In [1]:
# import dependencies
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
#from MulticoreTSNE import MulticoreTSNE as TSNE
import json
import base64
import zlib
from pyscenic.plotting import plot_binarization
from pyscenic.export import add_scenic_metadata
from pyscenic.cli.utils import load_signatures
import matplotlib as mpl
import matplotlib.pyplot as plt
#from scanpy.plotting._tools.scatterplots import plot_scatter
import seaborn as sns

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
from pyscenic.utils import load_motifs
import operator as op
from pyscenic.rss import regulon_specificity_scores
from IPython.display import HTML, display

In [3]:
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
#sc.logging.print_versions()
sc.settings.set_figure_params(dpi=150)

In [4]:
# loading DGE matrix
f_final_loom = '/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/pyscenic/filtered.loom'

In [5]:
# loading scenic run file
scenic_loom = '/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/pyscenic/SCENIC.loom'

In [6]:
# scenic output
lf = lp.connect( f_final_loom, mode='r', validate=False )

In [7]:
# scenic output
sc = lp.connect(scenic_loom, mode='r', validate=False )

In [10]:
# Seurat meta data loading
meta = pd.read_csv("/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/pyscenic/20220315_pyscenic_meta_Seurat_meta.csv")

In [12]:
exprMat = pd.DataFrame( sc[:,:], index=sc.ra.Gene, columns=sc.ca.CellID).T

In [13]:
auc_mtx = pd.DataFrame( sc.ca.RegulonsAUC, index=sc.ca.CellID)

In [14]:
auc_mtx.to_csv("/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/pyscenic/20220315_pyscenic_preprocessed_auc_mtx.csv")

In [15]:
auc_mtx_Z = pd.DataFrame( index=auc_mtx.index )
for col in list(auc_mtx.columns):
    auc_mtx_Z[ col ] = ( auc_mtx[col] - auc_mtx[col].mean()) / auc_mtx[col].std(ddof=0)

/scratch/tmp/ipykernel_57722/850879095.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  auc_mtx_Z[ col ] = ( auc_mtx[col] - auc_mtx[col].mean()) / auc_mtx[col].std(ddof=0)


In [16]:
auc_mtx_Z.to_csv("/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/pyscenic/20220315_pyscenic_preprocessed_auc_mtx_z_score.csv")

In [17]:
sig = load_signatures('/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/pyscenic/reg.csv')

Create regulons from a dataframe of enriched features.
Additional columns saved: []


In [18]:
# run by all cell type with exp separate
rss_cellType1 = regulon_specificity_scores( auc_mtx, meta['pyscenic1'] )
rss_cellType1

,Atf2(+),Atf3(+),Bach1(+),Bach2(+),Batf(+),Bcl3(+),Cebpb(+),Cebpg(+),Dlx1(+),Dlx5(+),...,Trp53(+),Usf1(+),Vsx2(+),Yy1(+),Zeb1(+),Zfp143(+),Zfp217(+),Zfp652(+),Zic1(+),Zmiz1(+)
Astrocytes_Ctrl,0.186660,0.210129,0.187031,0.191548,0.203630,0.189579,0.198215,0.194732,0.182870,0.184333,...,0.200767,0.193823,0.185352,0.198016,0.193035,0.184893,0.210046,0.180381,0.222352,0.180423
subiculum_Ctrl,0.235349,0.205359,0.240628,0.230930,0.183959,0.191463,0.215967,0.222085,0.189487,0.226780,...,0.215368,0.220388,0.227659,0.216173,0.229835,0.210993,0.192935,0.231523,0.195778,0.256785
MFOL_Ctrl,0.227896,0.243793,0.224345,0.235307,0.248815,0.306651,0.242296,0.241943,0.259257,0.234574,...,0.257403,0.244613,0.220284,0.252737,0.233416,0.239117,0.238511,0.192168,0.227237,0.184204
Unidentified_Neurons_Ctrl,0.179916,0.175657,0.177314,0.179354,0.175818,0.171799,0.181209,0.178249,0.171992,0.175765,...,0.175561,0.178099,0.175692,0.177805,0.178413,0.176390,0.175811,0.177300,0.171120,0.177723
CA1_Ctrl,0.223045,0.188495,0.214028,0.220512,0.186043,0.183864,0.205245,0.200508,0.181423,0.213118,...,0.205211,0.210009,0.217094,0.205505,0.219434,0.210218,0.185302,0.214674,0.179633,0.211133
Dentate_Gyrus_Ctrl,0.246667,0.242470,0.235959,0.237911,0.211286,0.190442,0.231477,0.229303,0.242341,0.232166,...,0.226591,0.235290,0.257368,0.226424,0.241878,0.240843,0.200146,0.235527,0.178926,0.266565
Inhibitory_Neurons_Ctrl,0.204107,0.187049,0.220461,0.203668,0.180471,0.183047,0.192369,0.197636,0.253414,0.236294,...,0.193588,0.200407,0.208365,0.196464,0.201134,0.196411,0.184299,0.202855,0.184523,0.204682
Microglia_Ctrl,0.186303,0.196074,0.184698,0.188093,0.208484,0.198407,0.204271,0.205587,0.185054,0.183907,...,0.195383,0.196979,0.185096,0.200318,0.188699,0.194562,0.238313,0.223756,0.179798,0.175401
MOL_Ctrl,0.190070,0.192996,0.187986,0.192468,0.186872,0.227365,0.192661,0.194646,0.196402,0.190290,...,0.202698,0.196013,0.186196,0.197249,0.192538,0.194356,0.193974,0.177182,0.203107,0.174971
CA2_3_Ctrl,0.195482,0.176275,0.188537,0.195302,0.197183,0.173201,0.185678,0.185483,0.174169,0.188852,...,0.188940,0.189920,0.193938,0.185310,0.193864,0.196260,0.175802,0.191726,0.173128,0.193830


In [19]:
# save
rss_cellType1.to_csv("/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/pyscenic/20220315_pyscenic_preprocessed__rss_cell_type_1st.csv")

In [20]:
# run by all cell type with exp separate
rss_cellType2 = regulon_specificity_scores( auc_mtx, meta['pyscenic2'] )

In [21]:
# save
rss_cellType2.to_csv("/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/pyscenic/20220315_pyscenic_preprocessed__rss_cell_type_2nd.csv")

In [22]:
# run by all cell type with exp separate
rss_cellType3 = regulon_specificity_scores( auc_mtx, meta['pyscenic3'] )

In [23]:
# save
rss_cellType3.to_csv("/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/pyscenic/20220315_pyscenic_preprocessed__rss_cell_type_3rd.csv")


In [24]:
# run by all cell type with exp separate
rss_cellType4 = regulon_specificity_scores( auc_mtx, meta['pyscenic4'] )

In [25]:
# save
rss_cellType4.to_csv("/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/pyscenic/20220315_pyscenic_preprocessed__rss_cell_type_4th.csv")

